In [ ]:
import sys
!{sys.executable} -m pip install xlsxwriter

In [ ]:
import pandas as pd                  # Manejo de datos tabulares
import numpy as np                   # Utilidades numéricas
import matplotlib.pyplot as plt      # Gráficos (sin seaborn)
from pathlib import Path             # Rutas multiplataforma
import unicodedata                   # Normalización unicode (acentos/NBSP)
import math                          # Funciones matemáticas (ceil)

In [ ]:
#Funciones
def strip_accents(s: str) -> str:
    # Elimina acentos/diacríticos para facilitar búsquedas por nombre de columna.
    return "".join(c for c in unicodedata.normalize("NFD", str(s)) if unicodedata.category(c) != "Mn")

def clean_colname(s: str) -> str:
    # Limpia espacios al inicio/fin y colapsa espacios múltiples a uno solo.
    s2 = str(s).strip()
    s2 = " ".join(s2.split())
    return s2

def to_float_comma(series: pd.Series) -> pd.Series:
    # Convierte números con coma decimal a float de Python (p. ej., "72,7" -> 72.7).
    return pd.to_numeric(
        series.astype(str).str.replace(".", "", regex=False).str.replace(",", ".", regex=False),
        errors="coerce"   # Si hay texto no convertible, lo pone como NaN.
    )

def clasificar_variable(serie: pd.Series) -> str:
    # Reglas para clasificar el tipo de variable a partir del dtype de pandas.
    if pd.api.types.is_datetime64_any_dtype(serie):
        return "Temporal (cuantitativa continua)"
    if pd.api.types.is_integer_dtype(serie):
        return "Cuantitativa discreta"
    if pd.api.types.is_float_dtype(serie):
        return "Cuantitativa continua"
    if pd.api.types.is_bool_dtype(serie):
        return "Cualitativa (binaria)"
    return "Cualitativa"  # objeto/categoría/strings

In [ ]:
# 0) Cargar y normalizar datos
RUTA = Path("Datos.csv")                                  # Cambia si tu archivo se llama distinto
if not RUTA.exists():
    RUTA = Path("/mnt/data/Tarea1.Datos-cuant-Gamma.csv") # Fallback (donde ya lo corrimos)

ENCODINGS = ["utf-8", "latin-1", "cp1252"]                # Codificaciones típicas en ES
SEPS      = [";", ",", "\t", "|"]                         # Separadores comunes

def read_csv_smart(path: Path) -> tuple[pd.DataFrame, str, str]:
    # Prueba combinaciones encoding/sep hasta cargar bien el CSV
    for enc in ENCODINGS:
        for sep in SEPS:
            try:
                df_try = pd.read_csv(path, encoding=enc, sep=sep)
                # Evita falso positivo: si queda 1 columna y se ven separadores, no era ese sep
                if df_try.shape[1] == 1 and any(s in str(df_try.iloc[0,0]) for s in [",",";","|","\t"]):
                    continue
                return df_try, enc, sep
            except Exception:
                continue
    raise ValueError("No fue posible leer el CSV con diferentes codificaciones/separadores.")

df, used_enc, used_sep = read_csv_smart(RUTA)             # ← Carga robusta
print(f"Leído con encoding='{used_enc}' sep='{used_sep}'. Cols: {list(df.columns)}")

df.columns = [clean_colname(c) for c in df.columns]       # Limpia nombres de columnas

# === Mapear nombres de columnas que usaremos (constantes) ===================
COL_CLIENTES_NUEVOS = "Clientes nuevos"
COL_RETENCION_PCT   = "Tasa de retención de clientes (%)"
COL_VISITAS         = "Número de visitas comerciales realizadas"
COL_VENTAS          = "Ventas totales"

# Detectar columna de satisfacción de forma robusta (con o sin acento)
COL_SATISFACCION = next(
    (c for c in df.columns if "satisfaccion" in strip_accents(c).lower()),
    None
)

# === Conversión de columnas con coma decimal a float ========================
if COL_RETENCION_PCT in df.columns:
    df[COL_RETENCION_PCT] = to_float_comma(df[COL_RETENCION_PCT])
if COL_SATISFACCION:
    df[COL_SATISFACCION] = to_float_comma(df[COL_SATISFACCION])

# === Forzar numéricos en columnas clave (enteras) ===========================
for col in [COL_CLIENTES_NUEVOS, COL_VISITAS, COL_VENTAS]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")   # Int64 (nullable)

# === Agregar columnas sintéticas SOLO si faltan =============================
np.random.seed(42)                                                          # Reproducible
n = len(df)
MES_NOMBRES = {1:"Enero",2:"Febrero",3:"Marzo",4:"Abril",5:"Mayo",6:"Junio",
               7:"Julio",8:"Agosto",9:"Septiembre",10:"Octubre",11:"Noviembre",12:"Diciembre"}

if "mes_num" not in df.columns:
    df["mes_num"] = np.tile(np.arange(1,13), math.ceil(n/12))[:n]
if "mes_nombre" not in df.columns:
    df["mes_nombre"] = df["mes_num"].map(MES_NOMBRES)
if "Asesor" not in df.columns:
    df["Asesor"] = np.tile(["Asesor_A","Asesor_B","Asesor_C"], math.ceil(n/3))[:n]
if "Ciudad" not in df.columns:
    df["Ciudad"] = np.tile(["Bogotá","Medellín","Cali"], math.ceil(n/3))[:n]

# (Opcional) Vista rápida
print(df.head(10))
print(df.dtypes)

In [ ]:
data= df
# 1) Cuadro de variables y clasificación Tarea a
tabla_variables = pd.DataFrame({
    "variable": data.columns,                                      # Nombre de columna.
    "tipo_inferido": [str(data[c].dtype) for c in data.columns],   # dtype de pandas (int64, float64, object, etc.).
    "clasificacion": [clasificar_variable(data[c]) for c in data.columns]  # Clasificación humana.
})
print("\n[1] Variables y clasificación")             # Imprime título de sección.
print(tabla_variables.to_string(index=False))        # Muestra la tabla de manera legible en consola.


In [ ]:
#2) Resumen numérico Tarea b
num_cols   = [c for c in data.columns if pd.api.types.is_numeric_dtype(data[c])]
resumen_num = (data[num_cols].agg(["mean","min","max","std"]).T
               .reset_index()
               .rename(columns={"index":"variable",
                                "mean":"promedio","min":"minimo",
                                "max":"maximo","std":"desviacion_estandar"}))
print("\n[2] Resumen numérico (promedio, min, max, sd)")
print(resumen_num.to_string(index=False))

In [ ]:


# 3) Total de ventas por mes Tarea c
ventas_mes = (data
              .groupby(["mes_num","mes_nombre"], as_index=False)[COL_VENTAS]
              .sum()
              .sort_values("mes_num"))
print(
    "\n[3] Total de ventas por mes\n",ventas_mes.to_string(index=False),f"\n\n[3] Total de ventas: {data[COL_VENTAS].sum()}")

In [ ]:
# 4) Tasa de conversión + promedio mensual por asesor + gráfico. Tarea D
#Tasa de conversión (Número de clientes nuevos)/(numero de visitas)*100
data["tasa_conversion_pct"] = np.where(
    data[COL_VISITAS].astype(float) > 0,
    (data[COL_CLIENTES_NUEVOS].astype(float) / data[COL_VISITAS].astype(float)) * 100.0,
    np.nan
)
conv_mensual_asesor = (data
                       .groupby(["Asesor","mes_num","mes_nombre"], as_index=False)["tasa_conversion_pct"]
                       .mean()
                       .sort_values(["mes_num","Asesor"]))
print("\n[4] Promedio mensual de tasa de conversión por asesor")
print(conv_mensual_asesor.to_string(index=False))

plt.figure()
for asesor, sub in conv_mensual_asesor.groupby("Asesor"):
    sub = sub.sort_values("mes_num")
    plt.plot(sub["mes_num"], sub["tasa_conversion_pct"], marker="o", label=str(asesor))
plt.xticks(ticks=range(1,13), labels=[MES_NOMBRES[i] for i in range(1,13)], rotation=45, ha="right")
plt.title("Promedio mensual de tasa de conversión por asesor")
plt.xlabel("Mes"); plt.ylabel("Tasa de conversión (%)")
plt.legend(loc="best"); plt.tight_layout(); plt.show()


In [ ]:
# 5) Box-plot de ventas. Tarea E
plt.figure()
caja = plt.boxplot(data[COL_VENTAS].dropna(), patch_artist=True)

# Título y ejes
plt.title("Box-plot de ventas totales")
plt.ylabel("Ventas")

# Etiquetas en los bigotes
limites = caja['whiskers']
for i, line in enumerate(limites):
    y = line.get_ydata()[1]
    plt.text(1.05, y, f"{y:.0f}", va='center')

# Extraer valores estadísticos
q1 = data[COL_VENTAS].quantile(0.25)
mediana = data[COL_VENTAS].median()
q3 = data[COL_VENTAS].quantile(0.75)

# Mostrar dentro de la caja
plt.text(1.05, q1, f"{q1:.0f}", va='center')
plt.text(1.05, mediana, f"{mediana:.0f}", va='center')
plt.text(1.05, q3, f"{q3:.0f}", va='center')

plt.tight_layout()
plt.show()



In [ ]:
# 6) Gráficos adicionales. Tarea f
# i) Ciclograma de ventas mensuales por asesor
angles = np.linspace(0, 2*np.pi, 12, endpoint=False)
ventas_mes_asesor = (data
                     .groupby(["Asesor","mes_num"], as_index=False)[COL_VENTAS]
                     .sum())
plt.figure()
ax = plt.subplot(111, polar=True)
for asesor, sub in ventas_mes_asesor.groupby("Asesor"):
    serie = pd.Series(index=range(1,13), dtype=float)
    serie[sub["mes_num"].values] = sub[COL_VENTAS].values
    serie = serie.fillna(0.0)
    vals = np.append(serie.values, serie.values[0])
    angs = np.append(angles, angles[0])
    ax.plot(angs, vals, marker="o", label=str(asesor))
ax.set_xticks(angles)
ax.set_xticklabels([MES_NOMBRES[i] for i in range(1,13)])
ax.set_title("Ciclograma de ventas mensuales por asesor")
ax.legend(loc="upper right", bbox_to_anchor=(1.2, 1.1))
plt.tight_layout(); plt.show()

# ii) Dispersión satisfacción vs ventas
# Asegura numérico y filtra filas válidas para evitar error con matplot
COL_SAT = "satisfaccion"  # nombre de la columna
data[COL_SAT] = pd.to_numeric(data[COL_SAT], errors="coerce")
mask = data[[COL_SAT, COL_VENTAS]].notna().all(axis=1)
df_scatter = data.loc[mask, [COL_SAT, COL_VENTAS]]

plt.figure()
plt.scatter(df_scatter[COL_SAT], df_scatter[COL_VENTAS], s=25, alpha=0.6)
plt.title("Satisfacción vs Ventas")
plt.xlabel("Satisfacción")
plt.ylabel("Ventas")
plt.grid(True, linestyle="--", alpha=0.3)
plt.tight_layout()
plt.show()

# Relación mensual: promedio de satisfacción vs. total de ventas por mes
rel_mensual = (
    data.groupby("mes_num", as_index=False)
        .agg(sat_prom=(COL_SAT, "mean"), ventas_tot=(COL_VENTAS, "sum"))
        .sort_values("mes_num")
)

plt.figure()
plt.scatter(rel_mensual["sat_prom"], rel_mensual["ventas_tot"], s=40, alpha=0.8)
for _, r in rel_mensual.iterrows():
    plt.annotate(str(MES_NOMBRES.get(int(r["mes_num"]), int(r["mes_num"]))),
                 (r["sat_prom"], r["ventas_tot"]), xytext=(4,4), textcoords="offset points", fontsize=8)
plt.title("Promedio de satisfacción vs ventas totales (por mes)")
plt.xlabel("Satisfacción promedio mensual")
plt.ylabel("Ventas totales mensuales")
plt.grid(True, linestyle="--", alpha=0.3)
plt.tight_layout()
plt.show()

# iii) Barras agrupadas: ventas mensuales por ciudad
pv = (
    data.groupby(["Ciudad", "mes_num"], as_index=False)[COL_VENTAS]
        .sum()
        .pivot(index="mes_num", columns="Ciudad", values=COL_VENTAS)
        .sort_index()
        .fillna(0.0)
)

# Gráfico de barras agrupadas
plt.figure()
idx = np.arange(len(pv.index))                  # posiciones por mes
n_series = max(1, pv.shape[1])                  # número de ciudades
width = 0.8 / n_series                          # ancho dinámico por barra

for i, ciudad in enumerate(pv.columns):
    y = pd.to_numeric(pv[ciudad], errors="coerce").fillna(0.0).to_numpy()
    plt.bar(idx + i*width, y, width=width, label=str(ciudad))

plt.xticks(idx + (n_series-1)*width/2,
           [MES_NOMBRES.get(m, str(m)) for m in pv.index],
           rotation=45, ha="right")

plt.title("Ventas mensuales por ciudad")
plt.xlabel("Mes")
plt.ylabel("Ventas")
plt.legend(loc="best")
plt.tight_layout()
plt.show()

# Tabla plana para exportar (mismo bloque)
Ventas_mensuales_por_ciudad = (
    pv.reset_index()
      .rename(columns={"mes_num": "Mes"})
      .assign(Mes_nombre=lambda d: d["Mes"].map(MES_NOMBRES))
      .loc[:, ["Mes", "Mes_nombre"] + list(pv.columns)]
)


In [ ]:
# 7) Dos cálculos adicionales (descriptiva). Tarea G
# A) Ventas promedio por visitas (promedio mensual) correlacion positiva leve 0.3018
data["Ventas promedio por visitas"] = np.where(
    data[COL_VISITAS].astype(float) > 0,
    data[COL_VENTAS].astype(float) / data[COL_VISITAS].astype(float),
    np.nan
)
Ventas_promedio_por_visitas = (data
                           .groupby(["mes_num","mes_nombre"], as_index=False)["Ventas promedio por visitas"]
                           .mean()
                           .sort_values("mes_num"))
print("\n[7A] Ventas promedio por visitas (promedio mensual)")
print(Ventas_promedio_por_visitas.to_string(index=False))

# B) Correlación Ventas vs Retención (%) correlacion positiva leve 0.3018 
# Se usa pearson porque ambas son numericas y continuas
# Se quiere medir la fuerza de la relacion lineal
if COL_RETENCION_PCT in data.columns:
    corr = np.corrcoef(data[COL_RETENCION_PCT].astype(float), data[COL_VENTAS].astype(float))[0,1]
    print("\n[7B] Correlación Pearson Ventas vs Retención (%):", round(float(corr), 4))
else:
    print("\n[7B] Aviso: no hay columna de retención para calcular correlación.")




In [ ]:
#C) Ventas por mes (ordenadas correctamente) ============================
if not {"mes_num","mes_nombre", COL_VENTAS}.issubset(df.columns):
    raise ValueError("Faltan columnas requeridas: 'mes_num', 'mes_nombre' o COL_VENTAS.")

ventas_mes = (df
              .groupby(["mes_num", "mes_nombre"], as_index=False)[COL_VENTAS]
              .sum()
              .sort_values("mes_num"))

# Crecimiento porcentual mes a mes ====================================
ventas_mes["%Δ Ventas"] = ventas_mes[COL_VENTAS].pct_change() * 100
print("\nVentas por mes y %Δ:")
print(ventas_mes)

# === 3) Gráfico de líneas: ventas totales por mes ===========================
plt.figure()
plt.plot(ventas_mes["mes_nombre"], ventas_mes[COL_VENTAS], marker="o")
plt.title("Ventas totales por mes")
plt.xlabel("Mes")
plt.ylabel("Ventas totales")
plt.grid(True)
# --- Soluciones para que no se peguen los meses ---
plt.xticks(rotation=45)        # gira las etiquetas 45° (puedes probar 60 o 90)
plt.tight_layout()             # ajusta márgenes automáticamente
plt.show()



In [ ]:
# 8) Exportación a Excel. Adicional.
# Asegura ruta y extensión .xlsx
out_dir = Path("salidas")
out_dir.mkdir(parents=True, exist_ok=True)
out_file = out_dir / "reporte.xlsx"

# Helper: escribir solo si el objeto existe y tiene datos
def write_df(writer, df_name, sheet):
    obj = globals().get(df_name, None)
    if isinstance(obj, pd.DataFrame) and not obj.empty:
        obj.to_excel(writer, sheet_name=sheet, index=False)
        print(f"✓ {sheet}: {obj.shape[0]} filas, {obj.shape[1]} cols")
    else:
        print(f"⚠ {sheet}: no se escribió (no existe o está vacío)")
   

with pd.ExcelWriter(out_file, engine="xlsxwriter") as writer:
    write_df(writer, "tabla_variables",       "Variables")
    write_df(writer, "resumen_num",           "ResumenNumerico")
    write_df(writer, "ventas_mes",            "VentasPorMes")
    write_df(writer, "conv_mensual_asesor",   "ConvMensualAsesor")
    write_df(writer, "Ventas_mensuales_por_ciudad",     "Ventas mensuales por ciudad")
    write_df(writer, "Ventas_promedio_por_visitas","Ventas promedio por visitas")

print(f"\n✅ Exportado: {out_file.resolve()}")